In [10]:
import pandas as pd
from datetime import date, timedelta
import urllib.request
import shutil
import os

In [8]:
# Batch download MTA turnstile files
start_date = date(2020, 1, 4)
end_date = date(2020, 4, 18)
while start_date <= end_date:
    filename = 'turnstile_'+start_date.strftime('%y%m%d')
    url = 'http://web.mta.info/developers/data/nyct/turnstile/'+filename+'.txt'
    print('Downloading '+filename+'.csv')
    with urllib.request.urlopen(url) as response, open('./data/'+filename+'.csv', 'wb') as out_file:
        shutil.copyfileobj(response, out_file)
    start_date += timedelta(weeks=1)

In [12]:
# Import station data with latitude and longitude
pd.options.display.float_format = '{:.6f}'.format
geocoded = pd.read_csv('geocoded.csv')

In [9]:
def filter(data):
    # Filter by regular records, remove PATH
    data_filtered = data[(data['DESC']=='REGULAR') | (data['DESC']=='RECOVR AUD')]
    data_filtered = data_filtered[data_filtered['DIVISION']!='PTH']

    # Get diff for each row
    data_filtered['ENTRIES_DIFF'] = data_filtered.groupby(['UNIT', 'SCP'])['ENTRIES'].diff()
    data_filtered['EXITS_DIFF'] = data_filtered.groupby(['UNIT', 'SCP'])['EXITS'].diff()
    data_filtered = data_filtered[(data_filtered['ENTRIES_DIFF']>=0) & (data_filtered['EXITS_DIFF']>=0)]

    # Remove some columns
    data_filtered.drop(columns=['STATION', 'LINENAME', 'DIVISION', 'DESC'], inplace=True)
    
    return data_filtered

In [13]:
# Batch process
directory = os.fsencode('./data')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    print('Cleaning '+filename)

    # Import MTA weekly turnstile data
    data = pd.read_csv('./data/'+filename, parse_dates=['DATE'])
    data.columns = data.columns.str.strip()
    
    data_filtered = filter(data)
    data_geocoded = pd.merge(data_filtered, geocoded, how='left', left_on=['UNIT'], right_on=['UNIT'])
    
    # Group and sum 
    data_cleaned = data_geocoded.groupby(['DATE', 'STATION', 'LATITUDE', 'LONGITUDE'])['ENTRIES_DIFF', 'EXITS_DIFF'].sum()
    data_cleaned = data_cleaned.astype(int)
    
    # Export
    data_cleaned.to_csv('./cleaned/cleaned_'+filename, float_format='%.6f')

Cleaning turnstile_200104.csv
Cleaning turnstile_200111.csv
Cleaning turnstile_200118.csv
Cleaning turnstile_200125.csv
Cleaning turnstile_200201.csv
Cleaning turnstile_200208.csv
Cleaning turnstile_200215.csv
Cleaning turnstile_200222.csv
Cleaning turnstile_200229.csv
Cleaning turnstile_200307.csv
Cleaning turnstile_200314.csv
Cleaning turnstile_200321.csv
Cleaning turnstile_200328.csv
Cleaning turnstile_200404.csv
Cleaning turnstile_200411.csv
Cleaning turnstile_200418.csv
Cleaning turnstile_200425.csv


In [89]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(data_cleaned)

ENTRIES_DIFF  EXITS_DIFF
DATE       STATION         LATITUDE  LONGITUDE                           
2020-04-18 1 AVE           40.730901 -73.981719          1014        2378
           103 ST          40.790582 -73.947473          1070        1544
                           40.796105 -73.961399           199         238
                           40.799354 -73.968329           602         701
           103 ST-CORONA   40.749858 -73.862672          1646        1458
           104 ST          40.695184 -73.844326           184          94
           110 ST          40.795066 -73.944297          1618        2336
           110 ST-CATHEDRL 40.804032 -73.966742          1190        1454
           110 ST-CPN      40.799110 -73.951807           487         847
           111 ST          40.697405 -73.836354           131          86
                           40.751760 -73.855183           948         669
           116 ST          40.798574 -73.941593          2472        3314
                           40.802098 -73.949625          1360        1520
                           40.805072 -73.954833           744        1344
           116 ST-COLUMBIA 40.808190 -73.964124           255         326
           121 ST          40.700536 -73.828382           155         124
           125 ST          40.804406 -73.937452          2556        4462
                           40.808076 -73.945906          1562        1762
                           40.811056 -73.952386          4134        5644
                           40.815596 -73.958395           403         855
           135 ST          40.814459 -73.940992          2096        1636
                           40.817902 -73.947644           732        1224
           137 ST-CITY COL 40.821994 -73.953674          1756        2548
           138 ST-3 AVE    40.810512 -73.926165           636         552
           138 ST-GR CONC  40.813208 -73.929877           332         384
           14 ST           40.738007 -73.998381          1270        1360
                           40.740388 -74.002104           717         905
           14 ST-6 AVE     40.737348 -73.996900          1384        3712
           14 ST-UNION SQ  40.734836 -73.990688         20358       28668
           145 ST          40.820402 -73.936315             4           3
                           40.824787 -73.944232          3484        4494
                           40.826426 -73.950412          1412        1868
           148 ST-LENOX    40.823877 -73.936443            15          61
           149 ST-3 AVE    40.816132 -73.917754          3506        2834
           149 ST-GR CONC  40.818429 -73.926927          2518        2704
           15 ST-PROSPECT  40.660376 -73.979509           346         372
           155 ST          40.829934 -73.938632           436         433
                           40.830551 -73.941486           174         225
           157 ST          40.833879 -73.944726          1494        1666
           161 ST-YANKEE   40.827888 -73.925736         10998        8586
           163 ST-AMSTERDM 40.835957 -73.939898           315         507
           167 ST          40.833773 -73.918430          1494         762
                           40.835535 -73.921380          1077        1400
           168 ST-BROADWAY 40.840778 -73.940091          4032        4146
           169 ST          40.710459 -73.793600           716        1244
           170 ST          40.839301 -73.913355          1286        1092
                           40.840048 -73.917775           828        1166
           174 ST          40.837382 -73.887659          1066        2050
           174-175 ST      40.845892 -73.910136           968         814
           175 ST          40.847369 -73.939683          1034        1040
           176 ST          40.848635 -73.912497           659         846
           18 AVE          40.607958 -74.001782           303         116
                           40.620728 -73.990388      